using example in https://huggingface.co/blog/how-to-train
https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=QDNgPls7_l13


feel free to edit/delete anything

In [1]:
import os
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

In [2]:
import transformers
import tokenizers

In [3]:
tokenizer = ByteLevelBPETokenizer(
    "./../tokenize/vocab.json",
    "./../tokenize/merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)
# or use the RobertaTokenizer from `transformers` directly.

examples = []

src_file = os.path.realpath("../tokenize/pre-tokenized/modified_cnn.txt")

with open(src_file, 'r', encoding="ISO-8859-1") as f:
        try:
            rawContent = f.read() 
        except Exception as e:
            print('During file reading, this error occurred:', e)


# tokenizer.encode(rawContent)
tokenizer.encode(rawContent).tokens

tokenizer.save("byte-level-BPE.tokenizer.json")

In [4]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)


In [5]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

model.num_parameters()

83504416

In [6]:
from transformers import LineByLineTextDataset
from transformers import PreTrainedTokenizerFast

# Debug through https://github.com/huggingface/transformers/issues/7234
tokenizer = PreTrainedTokenizerFast(tokenizer_file="byte-level-BPE.tokenizer.json")

# Just training on one point for debugging
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="../tokenize/pre-tokenized/modified_cnn.txt",
    block_size=128,
)


from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, mlm_probability=0.15
)

/Users/aaronxie/.local/share/virtualenvs/p-web-sfGkgXUp/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./model_one",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [8]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
# to check if everything is working later?
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./model_one",
    tokenizer="./model_one"
)


fill_mask("La suno <mask>.")